In [2]:
import pandas as pd
import seaborn as sns
from ipyleaflet import *
from branca.colormap import linear
import matplotlib.pyplot as plt
import json
import pandas
import folium
import geopandas as gpd
from colour import Color
from pandas_profiling import ProfileReport
import numpy as np
from ipywidgets import *
import sklearn as skl
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import explained_variance_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error 
from sklearn.metrics import mean_squared_log_error
from sklearn.metrics import median_absolute_error
from sklearn.metrics import r2_score
import warnings
warnings.filterwarnings("ignore")

In [3]:
# On importe le data frame que l'on a construit (cf note book get_data)

#df_regression = pd.read_csv(r'C:\Users\matth\OneDrive - GENES\ENSAE 2A\Python pour le data scientist/df_regression.csv')
df_regression = pd.read_csv('../df_regression.csv')


#On ne garde que les données à partir de 1995 pour que la régression soit la plus fiable possible
df_regression = df_regression[df_regression['year']>= 1995]

In [4]:
# On recalibre les données pour effectuer une régression, passage du format long au format wide
df_final = pd.DataFrame()
for cntry in df_regression.country.unique():
    df_temp = df_regression[df_regression.country == cntry]
    df_new = pd.DataFrame({cntry:[]})
    columns = np.array(df_temp.columns)
    for y in range(1990,2019):

        if df_temp[df_temp.year == y].empty:

            for col in columns:
                if col != 'year' and col != 'country':
                    filler = df_temp[col].mean()
                    df_new.loc[str(y) + '-' + col] = filler
        else:

            for col in columns:
                if col != 'year' and col != 'country':
                    df_new.loc[str(y) + '-' + col] = df_temp[df_temp.year == y][col].values

    df_final = pd.concat([df_final, df_new.transpose()])
    
df_final

,1990-croissancePIB,1990-mortalite-enf,1990-population,1990-export,1990-polity2,1990-ratio-zone,1990-ratio,1991-croissancePIB,1991-mortalite-enf,1991-population,...,2017-polity2,2017-ratio-zone,2017-ratio,2018-croissancePIB,2018-mortalite-enf,2018-population,2018-export,2018-polity2,2018-ratio-zone,2018-ratio
Zimbabwe,1.268572,20981.083333,1.256234e+07,136.330378,-1.416667,0.000450,0.000000,1.268572,20981.083333,1.256234e+07,...,4.0,0.000030,0.000000,4.824211,17413.0,14438812.0,210.771948,4.0,0.002011,0.000000
Zambia,5.534573,33454.041667,1.268306e+07,494.755586,5.000000,0.006294,0.000008,5.534573,33454.041667,1.268306e+07,...,6.0,0.000293,0.000000,4.034494,26866.0,17351714.0,1012.443792,6.0,0.002291,0.000000
Yemen,1.120798,40852.250000,2.128825e+07,111.755404,-1.166667,0.000316,0.000000,1.120798,40852.250000,2.128825e+07,...,0.0,0.000040,0.000000,0.752447,37517.0,28498683.0,33.288874,0.0,0.000128,0.000000
Vietnam,6.671504,31087.041667,8.523303e+07,500.452668,-7.000000,0.000000,0.000000,6.671504,31087.041667,8.523303e+07,...,-7.0,0.000000,0.000000,7.075789,25987.0,95545959.0,1682.653459,-7.0,0.000000,0.000000
Uzbekistan,5.771034,22790.041667,2.733835e+07,255.060565,-9.000000,0.132058,0.000215,5.771034,22790.041667,2.733835e+07,...,-9.0,0.199614,0.000000,5.354997,11479.0,32956100.0,387.671272,-9.0,0.312684,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Argentina,2.205179,10587.208333,3.954242e+07,180.912015,8.000000,0.003082,0.000000,2.205179,10587.208333,3.954242e+07,...,9.0,0.028273,0.000000,-2.617396,6560.0,44494502.0,234.543108,9.0,0.027312,0.000000
Angola,6.357488,82248.166667,2.124495e+07,381.830277,-2.958333,0.000045,0.006081,6.357488,82248.166667,2.124495e+07,...,-2.0,0.000000,0.000262,-2.003630,64019.0,30809787.0,514.553299,-2.0,0.000000,0.000279
Algeria,3.416667,21136.541667,3.457893e+07,175.686146,0.125000,0.010172,0.014457,3.416667,21136.541667,3.457893e+07,...,2.0,0.019693,0.000777,1.100000,20875.0,42228415.0,189.716555,2.0,0.024777,0.000407
Albania,4.837915,812.416667,2.994021e+06,435.138961,7.375000,0.000000,0.000003,4.837915,812.416667,2.994021e+06,...,9.0,0.000000,0.000000,4.019360,289.0,2866376.0,1097.548787,9.0,0.000000,0.000000


In [13]:
# On va essayer de prédire le ratio de risque de 2018 en utilisant toutes les données antérieures
df_final.rename(columns = {'2018-ratio':'aim'}, inplace = True)
df_final

# On sépare le jeu de données en entrainement / test
train, test = train_test_split(df_final, test_size = 0.2)
X_train = train.iloc[:, :-2]
y_train = train.iloc[:,-1:]
X_test = test.iloc[:, :-2]
y_test = test.iloc[:,-1:]

# On standardise les valeurs passées en entrée
scalerX = StandardScaler().fit(X_train)
scalerX_test = StandardScaler().fit(X_test)

X_strain = scalerX.transform(X_train)
X_stest = scalerX_test.transform(X_test)

In [14]:
# On effectue une régression linéaire utilisant toutes les données depuis 1995 pour prédire la dangerosité du pays en 2018

clf = LinearRegression()
clf.fit(X_strain, y_train.values)

# On regarde les prédictions
X_test['aim'] = y_test
X_test['prediction'] = clf.predict(X_stest)

In [15]:
X_test['country'] = X_test.index.values
X_test

,1990-croissancePIB,1990-mortalite-enf,1990-population,1990-export,1990-polity2,1990-ratio-zone,1990-ratio,1991-croissancePIB,1991-mortalite-enf,1991-population,...,2017-ratio-zone,2017-ratio,2018-croissancePIB,2018-mortalite-enf,2018-population,2018-export,2018-polity2,aim,prediction,country
Suriname,2.881175,256.208333,5.089002e+05,322.821302,5.000000,0.003090,0.000000,2.881175,256.208333,5.089002e+05,...,0.028273,0.000000,4.948271,177.0,575987.0,537.325863,5.0,0.000000,0.033358,Suriname
Costa Rica,4.190226,729.541667,4.342694e+06,135.989294,10.000000,0.000000,0.000000,4.190226,729.541667,4.342694e+06,...,0.000000,0.000000,2.615904,528.0,4999443.0,191.483642,10.0,0.000000,0.020038,Costa Rica
Honduras,3.943049,5160.750000,7.688281e+06,161.803018,6.833333,0.000329,0.000233,3.943049,5160.750000,7.688281e+06,...,0.000000,0.000000,3.844995,3082.0,9587523.0,256.849699,7.0,0.000000,-0.029897,Honduras
Oman,3.467208,792.958333,3.005326e+06,221.374709,-8.291667,0.000316,0.000000,3.467208,792.958333,3.005326e+06,...,0.000040,0.000000,1.245505,889.0,4829476.0,368.958846,-8.0,0.000000,0.006925,Oman
Lebanon,3.840818,1061.041667,4.908693e+06,405.731589,5.833333,0.152143,0.001963,3.840818,1061.041667,4.908693e+06,...,0.338424,0.002998,-1.702175,742.0,6859408.0,535.957905,6.0,0.000023,-0.162486,Lebanon
Portugal,1.517624,435.750000,1.037744e+07,177.018836,10.000000,0.000302,0.000000,1.517624,435.750000,1.037744e+07,...,0.000162,0.000000,2.849326,244.0,10283822.0,281.284272,10.0,0.000000,0.025138,Portugal
Turkmenistan,6.950247,5492.375000,4.931502e+06,272.979931,-8.750000,0.130985,0.000000,6.950247,5492.375000,4.931502e+06,...,0.200200,0.000000,6.200000,5104.0,5850902.0,381.996409,-8.0,0.000000,-0.015507,Turkmenistan
Austria,1.884687,323.750000,8.289963e+06,184.434681,10.000000,0.000005,0.000000,1.884687,323.750000,8.289963e+06,...,0.000000,0.000000,2.501595,254.0,8840521.0,273.624564,10.0,0.000000,0.021543,Austria
Norway,2.139604,185.916667,4.767439e+06,164.740430,10.000000,0.000002,0.000000,2.139604,185.916667,4.767439e+06,...,0.000050,0.000000,1.118829,120.0,5311916.0,202.789804,10.0,0.000000,0.017777,Norway
Israel,3.968635,617.708333,7.167554e+06,145.451918,6.000000,0.148847,0.007485,3.968635,617.708333,7.167554e+06,...,0.350274,0.000252,3.983425,512.0,8882800.0,197.271543,6.0,0.000884,-0.233593,Israel


In [16]:
# On récupére une base de donnée comprenant une liste de données sur les pays du monde
countries = gpd.read_file(gpd.datasets.get_path("naturalearth_lowres"))

In [17]:
# on crée une carte vierge 
m_predict = folium.Map(tiles='CartoDB Positron',attr=None,zoom_start=10)
# on genère la carte à deux couches  
folium.Choropleth(
  geo_data=countries[countries.name.isin(X_test.index.values)].to_json(),
    data=X_test,
    columns=['country','prediction'],
    key_on="properties.name", 
     fill_color="YlOrRd",
    fill_opacity=0.7,
    line_opacity=0,
    smooth_factor=0).add_to(m_predict)

m_aim = folium.Map(tiles='CartoDB Positron',attr=None,zoom_start=4)
# on genère la carte à deux couches  
folium.Choropleth(
  geo_data=countries[countries.name.isin(X_test.index.values)].to_json(),
    data=X_test,
    columns=['country','aim'],
    key_on="properties.name", 
     fill_color="YlOrRd",
    fill_opacity=0.7,
    line_opacity=0,
    smooth_factor=0).add_to(m_aim)

In [18]:
m_predict

In [19]:
m_aim